<a href="https://colab.research.google.com/github/Sriharsha0908/Sriharsha0908/blob/main/FMML_M1L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [1]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [3]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

:Number of Instances: 20640

:Number of Attributes: 8 numeric, predictive attributes and the target

:Attribute Information:
    - MedInc        median income in block group
    - HouseAge      median house age in block group
    - AveRooms      average number of rooms per household
    - AveBedrms     average number of bedrooms per household
    - Population    block group population
    - AveOccup      average number of household members
    - Latitude      block group latitude
    - Longitude     block group longitude

:Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived from the 1990 U.S. census, using one row per ce

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [4]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [5]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [6]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [7]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [8]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [9]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [10]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [11]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [12]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [13]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [14]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [17]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [18]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.

1A:

### 1. **Reduces Variance**
   - In a single train-test split, the validation accuracy can vary significantly depending on how the data is split. If the split happens to include an unusual distribution of classes or data points, it can give an unrepresentative estimate of the model's performance.
   - By averaging the accuracy across multiple splits, the model is tested on different subsets of the data, reducing the impact of any particular outlier or unusual data distribution, and hence producing a more **stable estimate** of the model's true performance.

### 2. **More Comprehensive Testing**
   - **k-fold cross-validation** ensures that every data point is used for both training and validation (though in different folds), meaning that the model gets exposed to all parts of the data. This reduces the likelihood that the model’s performance is skewed by a biased split or by not encountering important patterns in the training set.
   - This ensures the model is tested under various conditions, making the evaluation more **robust**.

### 3. **Prevents Overfitting to a Specific Split**
   - If you rely on a single validation split, the model may perform particularly well or poorly based on the random allocation of training and validation data. This can lead to overfitting on a specific set of data or underestimating performance due to an unfavorable validation set.
   - By averaging across splits, you reduce the risk of overfitting to any one particular validation set.

### 4. **More Reliable for Model Comparison**
   - When comparing different models or hyperparameter settings, averaging accuracy across multiple splits gives a better picture of how each model performs across a range of data distributions. This makes it easier to identify the model that generalizes the best, rather than one that might have performed well on a lucky validation split.
   
### 5. **Reduced Sensitivity to Data Order**
   - Some algorithms can be sensitive to the order in which data is provided. By shuffling and splitting the data into multiple folds, you can mitigate the risk that any single validation split disproportionately affects the results due to the data order.

### When Does Averaging Across Multiple Splits Help the Most?
- **Small Datasets**: On small datasets, a single train-test split can lead to high variance in performance. Cross-validation (and averaging accuracies) is especially helpful here, as it ensures all data points are used for training and validation.
- **Class Imbalance**: If you have imbalanced classes, a single split might not evenly distribute the classes between training and validation sets. Cross-validation helps to ensure that the evaluation is more balanced.
- **Noisy Data**: If your dataset contains noise or outliers, the performance of a model may vary depending on whether these noisy data points end up in the training or validation set. Averaging across splits smooths out these effects.

### Limitations of Averaging Validation Accuracy
- **Computational Cost**: Running multiple splits (especially in **k-fold cross-validation**) can be computationally expensive, particularly with large datasets or complex models. However, this trade-off is often worth it for the increased reliability.
- **Data Leakage**: Care must be taken to ensure that no information from the validation set leaks into the training set during cross-validation. This could occur if preprocessing is not done correctly (e.g., normalizing data before splitting).

### Conclusion
Averaging the validation accuracy across multiple splits (through techniques like cross-validation) generally provides **more consistent, reliable, and unbiased results**. It helps reduce the impact of random chance associated with any single data split, making it a preferred method for model evaluation, especially in scenarios where data is limited or the model is prone to overfitting.

2A:

### 1. **Reduces Bias**
   - In a single train-test split, the validation accuracy can be influenced by the specific selection of data points in the training and test sets. This split might not be representative of the entire dataset, leading to either an optimistic or pessimistic estimate of the model's performance.
   - By using multiple splits and averaging the accuracy, you reduce the bias that comes from an unrepresentative single split. Each fold gives the model a chance to train on different portions of the data, resulting in a more balanced and accurate reflection of the model's performance on unseen data.

### 2. **Simulates Multiple Test Scenarios**
   - Each fold in **k-fold cross-validation** can be seen as a simulation of a different test scenario. Since the model is evaluated on different subsets of the data in each fold, the averaged validation accuracy provides a more realistic estimate of how the model will perform on truly unseen test data, accounting for various possible data distributions.
   - The **variance** of the test set distribution is captured, making the average validation accuracy a better predictor of the actual test performance.

### 3. **Uses More Data for Training**
   - In each fold, a different subset of the data is used for validation, while the rest is used for training. This means that over the course of the cross-validation, **all data points** are eventually used for both training and validation.
   - This leads to a more **comprehensive utilization of the dataset**, reducing the likelihood that important patterns are missed during training, and thus improving the model’s ability to generalize to new data.

### 4. **Smoothed Out Performance Estimates**
   - Averaging over multiple folds helps smooth out fluctuations in performance that may be caused by chance. For example, a model might perform unusually well on one fold because the validation set is easier, or poorly on another due to difficult or outlier data points. Averaging these results gives a more **consistent and reliable** estimate of how well the model would perform on a truly unseen test set.

### 5. **Improves Estimate of Generalization Error**
   - Generalization error refers to the difference in performance between training data and unseen test data. **k-fold cross-validation** provides a better estimate of the generalization error because it forces the model to learn and be validated on different portions of the data, mimicking how the model might generalize to unseen test data in practice.
   - Averaging across folds smooths out noise and variability that could distort the estimate in a single train-test split.

### 6. **Helps with Class Imbalance**
   - If your dataset has an imbalance between classes, a single train-test split might not have an even distribution of classes in the validation set, leading to a misleading accuracy estimate. **k-fold cross-validation** ensures that each class gets represented in multiple splits, giving a more accurate estimate of the model's ability to generalize across all classes.

### 7. **Minimizes Overfitting to Specific Splits**
   - If a model is overfitting to a specific training set (as might happen in a single train-test split), the validation accuracy might be artificially high, leading to an inaccurate estimate of the test accuracy. By using multiple splits, you reduce the risk of overfitting to any one particular validation set and get a more accurate view of how the model will perform on unseen data.

### Important Considerations
- **Training vs Test Data Leakage**: While cross-validation provides a more accurate estimate of test accuracy, it's essential to ensure that there is no data leakage (such as normalizing data before splitting or using future information in a time series). Otherwise, the accuracy estimate will be overly optimistic and unreliable.
- **Hyperparameter Tuning**: If you're using cross-validation to tune hyperparameters, the final estimate of test accuracy should come from a completely **separate test set** that was not used during the cross-validation or tuning process. This prevents overfitting to the cross-validation process itself.

### Conclusion
Averaging the validation accuracy across multiple splits does indeed provide a more **accurate and reliable estimate of test accuracy** than relying on a single train-test split. It reduces variance, bias, and overfitting while utilizing more data for training, leading to a better reflection of how the model will perform on unseen test data.

3A:

1. **Convergence to True Value**: As the number of iterations increases, the estimate tends to converge toward the true value, assuming the algorithm is designed properly. Each iteration adds more information or refines the approximation, reducing the impact of noise or randomness.

2. **Reduction in Variance**: With more iterations, the variance of the estimate usually decreases. This means the estimate becomes more stable and less sensitive to random fluctuations or outliers.

3. **Law of Large Numbers**: In the case of simulations like Monte Carlo, the law of large numbers suggests that as the number of iterations (or trials) increases, the average of the results becomes closer to the expected value.

4. **Better Exploration of the Solution Space**: In optimization algorithms, more iterations allow the algorithm to explore the solution space more thoroughly, reducing the risk of being trapped in local minima and moving closer to a global optimum.

### Trade-offs:
- **Diminishing Returns**: While more iterations generally improve the estimate, there is a point of diminishing returns where additional iterations offer very marginal improvements.
- **Computational Cost**: Higher iterations also mean increased computational time and resource usage, so there’s a balance between the quality of the estimate and the time available to compute it.

In summary, higher iterations typically provide better estimates but come with computational trade-offs, and the rate of improvement may slow down after a certain point.

4A:

### 1. **Overfitting on Small Training Datasets**:
   - **More iterations** on a very small training dataset can lead to overfitting, where the model learns the noise and specific details of the small dataset rather than general patterns that apply to unseen data. The model might perform very well on the training set but poorly on the test or validation set.
   - Increasing iterations doesn’t increase the variety of examples, so the model may fail to generalize well to new, unseen data.

### 2. **Insufficient Generalization**:
   - With a **small training set**, the model lacks enough information about the underlying distribution of the data. No matter how many iterations you use, the model will still be trained on a limited and potentially biased set of examples, resulting in poor generalization to real-world data or unseen samples.
   
### 3. **Validation Dataset Problems**:
   - If the **validation dataset** is too small, increasing iterations may give misleading performance metrics. A small validation set can lead to high variance in evaluation results, making it difficult to assess whether the model is truly improving or just performing well by chance on the small set of validation examples.

### 4. **Bias-Variance Tradeoff**:
   - A small dataset introduces high variance (large fluctuations between different datasets) and potentially high bias (if the data does not capture the underlying complexity of the problem). Increasing iterations doesn’t address this issue; it may reduce variance but will not reduce bias caused by the small dataset size.

### 5. **Solutions**:
   - **Data Augmentation**: For small datasets, you can artificially expand the training data through techniques like data augmentation, especially in domains like image processing.
   - **Regularization**: Use regularization techniques (like L2 regularization, dropout, or early stopping) to prevent overfitting, especially with small datasets.
   - **Transfer Learning**: Instead of training from scratch on a small dataset, you can use a pre-trained model and fine-tune it on your small dataset.
   - **Cross-validation**: Instead of a fixed validation set, use cross-validation to make better use of the limited data available and get a more reliable estimate of model performance.

In summary, increasing iterations cannot compensate for a small training or validation dataset. Data quantity and quality are key factors in model performance, and other strategies like data augmentation, transfer learning, and regularization are more effective ways to address small datasets.